In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [ ]:
def import_data(file_name):
    path = "../resources/test_csv/"
    test_df = pd.read_csv(os.path.join(path, file_name))
    fnames = test_df["fname"]
    test_df.drop(labels="fname", axis='columns', inplace=True)
    features = test_df.values.reshape(-1, 129, 71, 1)
    return (features, fnames)

In [ ]:
def submit(model_array, models_path, features, fnames, classes, threshold=0.6):
    if (type(threshold) == int):
        threshold = [threshold] * len(model_array)
    for i in range(len(model_array)):
        print("loading model {} - {}/{}".format(model_array[i], i, len(model_array)))
        model = tf.keras.models.load_model(os.path.join(models_path, model_array[i] + ".h5"))
        preds = model.predict(features, batch_size=256, verbose=1)
        max_ind = np.argmax(preds, axis=1)
        max_val = preds.max(axis=1)
        predictions = np.where(max_val < threshold[i], 11, max_ind)
        predictions = np.vectorize(classes.get)(predictions)
        print("writing to file...")
        submission = pd.DataFrame({"fname": fnames, "label": predictions})
        submission_name = "submissions/submission_{}_t_{}.csv".format(model_array[i], threshold[i])
        if (os.path.isfile(submission_name)):
            submission.to_csv(submission_name,
                              header=False, mode='a', index=False)
        else:
            submission.to_csv(submission_name, index=False)

In [14]:
#file_array = next(os.walk("../resources/test_csv/"))[2]
model_array = [] # list of models
models_path = "../models/saved_models"
classes = {0: "yes", 1: "no", 2: "up", 3: "down", 4: "left", 5: "right", 6: "on",
           7: "off", 8: "stop", 9: "go", 10: "silence", 11: "unknown"}
threshold = 0.7

In [ ]:
%%time
for i in range(8):
    print("importing data... {} out of {}".format(i + 1, 8))
    features, fnames = import_data(file_array[i])
    print("finished importing")
    submit(model_array, models_path, features, fnames, classes, threshold)
    %reset_selective -f features
    %reset_selective -f fnames